<a href="https://colab.research.google.com/github/MaulidinZakaria/16_MaulidinZakaria-Machine_Learning/blob/main/Pertemuan_4/tugas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Import Library**

In [ ]:
# Import required libraries
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
from zipfile import ZipFile
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

1. Pathlib.Path: Menyediakan cara berorientasi objek untuk menangani dan memanipulasi jalur file.
2. cv2: Library OpenCV yang digunakan untuk pemrosesan gambar, seperti membaca gambar, mengubah ruang warna, dan menghitung histogram.
3. numpy (np): Digunakan untuk operasi numerik seperti array dan matriks, penting untuk mengelola data gambar.
4. pandas (pd): Untuk manipulasi dan analisis data. Dalam kode ini tidak digunakan secara eksplisit tetapi mungkin diperlukan dalam analisis dataset yang lebih besar.
5. ZipFile: Dari modul zipfile di Python, digunakan untuk mengekstrak data dari arsip ZIP, dalam hal ini adalah sekumpulan gambar.
6. SVC (Support Vector Classifier): Algoritma Support Vector Machine dari sklearn yang digunakan untuk tugas klasifikasi.
7. accuracy_score: Fungsi dari sklearn untuk menghitung akurasi prediksi model.
8. GridSearchCV: Digunakan untuk mencari secara menyeluruh nilai hyperparameter terbaik untuk meningkatkan performa model.
9. StandardScaler: Untuk menormalkan fitur dengan menghilangkan rata-rata dan menskalakan ke variansi satuan, yang dapat membantu kinerja model SVM.

## **2. Load Data**

In [ ]:
# Load and extract the dataset from zip file
with ZipFile('images.zip', 'r') as zip:
    zip.extractall()
    print('Dataset extracted!')

# Image directories
train_dir = "images/training/"
test_dir = "images/test/"

# Function to load images and labels
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')
    img_list = []
    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = cv2.imread(str(file))
            if img is not None:
                img_list.append((img, label))
    return img_list

# Load the training and test data
train_img = load_dataset(train_dir)
test_img = load_dataset(test_dir)

Dataset extracted!


1. Bagian ini mengekstrak dataset dari file ZIP bernama images.zip ke direktori saat ini sehingga data gambar dapat diakses untuk proses selanjutnya.

2. Bagian ini mendefinisikan jalur direktori yang berisi gambar-gambar pelatihan (training) dan pengujian (test).

3. Fungsi load_dataset memuat dataset gambar dari direktori tertentu, membaca gambar-gambar tersebut, mengasosiasikan gambar dengan label (nama sub-direktori tempat gambar berada), dan mengembalikannya sebagai list berisi tuple (gambar, label).


## **3. Pra Pengolahan Data**

In [ ]:
# Function to standardize the input image
def standarized_input(image, size=(1100, 600)):  # Reduce size to minimize feature complexity
    std_img = cv2.resize(image, size)
    return std_img

# Preprocess the image and label
def preprocess(img_list):
    std_img_list = []
    for item in img_list:
        image = standarized_input(item[0])  # Standardize image
        label = 1 if item[1] == 'day' else 0  # Encode label
        std_img_list.append((image, label))
    return std_img_list

train_std_img_list = preprocess(train_img)
test_std_img_list = preprocess(test_img)

1. standarized_input berfungsi untuk menyamakan ukuran gambar agar dataset memiliki dimensi yang konsisten.
2. preprocess mengubah ukuran gambar dan menyandikan label gambar ke dalam format numerik, sehingga data siap digunakan untuk pelatihan atau pengujian model machine learning.

## **4. Ekstrasi Fitur dengan Histogram**

In [ ]:
# Function to calculate color histogram for each image using fewer bins to reduce complexity
def calc_histogram(image):
    # Convert to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Calculate histogram for all three channels H, S, and V using fewer bins
    hist_h = cv2.calcHist([hsv_image], [0], None, [20], [0, 180])  # Reduce bins for Hue
    hist_s = cv2.calcHist([hsv_image], [1], None, [20], [0, 256])  # Reduce bins for Saturation
    hist_v = cv2.calcHist([hsv_image], [2], None, [20], [0, 256])  # Reduce bins for Value

    # Normalize the histograms
    hist_h = cv2.normalize(hist_h, hist_h).flatten()
    hist_s = cv2.normalize(hist_s, hist_s).flatten()
    hist_v = cv2.normalize(hist_v, hist_v).flatten()

    # Concatenate all histograms into one feature vector
    hist_features = np.concatenate([hist_h, hist_s, hist_v])

    return hist_features

# Extract histogram features and labels for the dataset
def extract_histogram_features(img_list):
    features = []
    labels = []
    for item in img_list:
        hist = calc_histogram(item[0])  # Extract histogram feature
        features.append(hist)
        labels.append(item[1])  # Label: day=1, night=0
    return np.array(features), np.array(labels)

# Extract features for train and test set
X_train, y_train = extract_histogram_features(train_std_img_list)
X_test, y_test = extract_histogram_features(test_std_img_list)

1. Fungsi calc_histogram digunakan untuk menghitung dan mengekstrak histogram warna dari tiap gambar dalam ruang warna HSV, dan hasilnya digunakan sebagai fitur input untuk model machine learning.
2. Fungsi extract_histogram_features mengambil daftar gambar dan label, lalu mengekstrak fitur histogram dari gambar tersebut untuk digunakan dalam pelatihan atau pengujian model.

## **5. Normalisasi Fitur dan Tuning Hyperparameter**

In [ ]:
# Normalize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modify hyperparameter tuning to reduce overfitting
param_grid = {
    'C': [0.001, 0.01, 0.1],  # Smaller C values to reduce overfitting
    'gamma': [0.1, 0.01, 0.001],  # Test smaller gamma to balance generalization
    'kernel': ['rbf']  # Using RBF kernel
}

svc = SVC()

# Grid search to find the best hyperparameters
grid_search = GridSearchCV(svc, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters from grid search
print(f'Best parameters found: {grid_search.best_params_}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters found: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}


1. Menormalkan data agar model SVC bisa bekerja lebih baik dengan data fitur yang terdistribusi secara konsisten.
2. Mencari kombinasi hyperparameter terbaik untuk model SVC dengan menggunakan grid search dan cross-validation, sehingga model tidak overfit dan bisa bergeneralisasi dengan baik ke data baru

## **6. Evaluasi Model**

In [ ]:
# Train the model with the best hyperparameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Make predictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print evaluation results
print(f'Accuracy on train data: {train_accuracy * 100:.2f}%')
print(f'Accuracy on test data: {test_accuracy * 100:.2f}%')


Accuracy on train data: 98.75%
Accuracy on test data: 96.25%


1. Pelatihan model dilakukan dengan hyperparameter terbaik yang dihasilkan dari grid search.
2. Model melakukan prediksi pada data latih dan data uji.
3. Akurasi pada kedua dataset dihitung dan ditampilkan, yang memberikan gambaran tentang seberapa baik model bekerja di kedua situasi tersebut: pelatihan (menghafal pola data) dan pengujian (kemampuan generalisasi ke data baru).